In [1]:
import numpy as np

# Part 1

In [2]:
digits = {0:['abcefg', 6],
          1:['cf',     2],
          2:['acdeg',  5],
          3:['acdfg',  5],
          4:['bcdf',   4],
          5:['abdfg',  5],
          6:['abdefg', 6],
          7:['acf',    3],
          8:['abcdefg',7],
          9:['abcdfg', 6]}

sgment = {'abcefg': '0',
          'cf':     '1',
          'acdeg':  '2',
          'acdfg':  '3',
          'bcdf':   '4',
          'abdfg':  '5',
          'abdefg': '6',
          'acf':    '7',
          'abcdefg':'8',
          'abcdfg': '9'}

In [3]:
tst = np.array(['acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf'])

test = np.array(['be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe',
                 'edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc',
                 'fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg',
                 'fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb',
                 'aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea',
                 'fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb',
                 'dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe',
                 'bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef',
                 'egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb',
                 'gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce'])

In [4]:
def split_in_out(data):
    out = []
    bot = []
    for line in data:
        i, o = line.split(' | ')
        line_in = i.split(' ')
        line_out = o.split(' ')
        out.append(line_out)
        b = line_in + line_out
        bot.append(b)
    return out, np.array(bot)

def count_unique(data):
    count = 0
    for i in range(0, len(data)):
        for j in range(0, len(data[i])):
            print
            if len(data[i][j]) == digits[1][1] or len(data[i][j]) == digits[4][1] or\
                len(data[i][j]) == digits[7][1] or len(data[i][j]) == digits[8][1]:
                count += 1
    return count

def part1(data):
    out, _ = split_in_out(data)
    return count_unique(out)

print(part1(test))

26


In [5]:
inpt = np.genfromtxt('day8_input.txt', dtype=str, delimiter='\n')
print('Part 1 Result:', part1(inpt))

Part 1 Result: 245


# Part 2

In [6]:
def get_lengths(data):
    length = []
    for dat in data:
        length.append(len(dat))
    return np.array(length)

def find_char_idx(char, strings):
    idx = []
    for i in range(0, len(strings)):
        if char in strings[i]:
            idx.append(i)
    return np.array(idx)

def get_mapping(bot):
    mapping = {'a':'', 'b':'', 'c':'', 'd':'', 'e':'', 'f':'', 'g':''}
    #find 1 and 7 as it will give the mapping to a
    lens = get_lengths(bot)
    one = np.where(lens == digits[1][1])[0]
    seven = np.where(lens == digits[7][1])[0]
    
    #a mapping is the char in seven but not in one
    for c in bot[seven[0]]:
        if c not in bot[one[0]]:
            mapping['a'] = c
        else:
            mapping['c'] += c
            mapping['f'] += c

    #3 will be only len(5) with c and f
    #Find 3
    five_seg = np.where(lens == 5)[0]
    one_0 = find_char_idx(mapping['c'][0],bot)
    one_1 = find_char_idx(mapping['c'][1],bot)
    one_edge = np.intersect1d(one_0, one_1)
    three = np.intersect1d(five_seg, one_edge)

    #Find 4
    four = np.where(lens == digits[4][1])[0]

    #With 3 and 4 and "knowing" 'cf', can get b and d
    for c in bot[four[0]]:
        if c in mapping['c']:
            continue
        elif c in bot[three[0]]:
            mapping['d'] = c
        else:
            mapping['b'] = c

    #With 3 and 4 and "knowing" 'cf' and knowing b and d, can get g
    for c in bot[three[0]]:
        if c in mapping['c'] or c == mapping['a'] or c == mapping['d']:
            continue
        else:
            mapping['g'] = c
            break

    #Only don't have any idea about e, so find it
    for c in mapping.keys():
        if c == mapping['a'] or c == mapping['b'] or c in mapping['c'] or\
            c == mapping['d'] or c == mapping['g']:
            continue
        mapping['e'] = c
        break

    #Can now get c and f from 2 or 5
    #Find 2
    two_a = find_char_idx(mapping['a'],bot)
    two_d = find_char_idx(mapping['d'],bot)
    two_e = find_char_idx(mapping['e'],bot)
    two_g = find_char_idx(mapping['g'],bot)
    two = np.intersect1d(two_a, two_d)
    two = np.intersect1d(two, two_e)
    two = np.intersect1d(two, two_g)
    two = np.intersect1d(two, five_seg)

    for c in bot[two[0]]:
        if c == mapping['a'] or c == mapping['d'] or c in mapping['e'] or\
            c == mapping['g']:
            continue
        mapping['c'] = c
        mapping['f'] = mapping['f'].replace(c, "")
        break
    
    inv_mapping = {v: k for k, v in mapping.items()}
    return inv_mapping

def decode_output(mapping, output):
    value = ''
    for o in output:
        decoded = ''
        for c in o:
            decoded += mapping[c]
        decoded = "". join(sorted(decoded))
        value += sgment[decoded]
    return int(value)

def part2(data):
    values = []
    out, bot = split_in_out(data)
    for i in range(0, len(bot)):
        mapping = get_mapping(bot[i])
        values.append(decode_output(mapping, out[i]))
    return np.sum(values)

print(part2(tst))
print(part2(test))

5353
61229


In [7]:
print('Part 2 Result:', part2(inpt))

Part 2 Result: 983026
